In [9]:
from helpers import *

import shutil


import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


from collections import OrderedDict
from itertools import product



linestyles = OrderedDict(
    [

     ('loosely dashed',      (0, (5, 20))),
     ('dashed',              (0, (3, 2))),
     ('densely dashed',      (0, (1, 3))),

    ])

# Data Processing

## Importing data

In [10]:
def load_kaggle_data(path):
    data = pd.read_csv(path)
    data.replace({'\\N':np.nan}, inplace=True)
    return data

data_dir ='kaggle-data/'

circuits_raw_df = load_kaggle_data(f'{data_dir}/circuits.csv')
constructors_raw_df = load_kaggle_data(f'{data_dir}/constructors.csv')
constructor_standings_raw_df = load_kaggle_data(f'{data_dir}/constructor_standings.csv')
driver_standings_raw_df = load_kaggle_data(f'{data_dir}/driver_standings.csv')
drivers_raw_df = load_kaggle_data(f'{data_dir}/drivers.csv')
laptimes_raw_df = load_kaggle_data(f'{data_dir}/lap_times.csv')
pitstops_raw_df = load_kaggle_data(f'{data_dir}/pit_stops.csv')
qualifying_raw_df = load_kaggle_data(f'{data_dir}/qualifying.csv')
races_raw_df = load_kaggle_data(f'{data_dir}/races.csv')
results_raw_df = load_kaggle_data(f'{data_dir}/results.csv')
seasons_raw_df = load_kaggle_data(f'{data_dir}/seasons.csv')
sprint_results = load_kaggle_data(f'{data_dir}/sprint_results.csv')
status_raw_df = load_kaggle_data(f'{data_dir}/status.csv')

print('circuits_raw_df', circuits_raw_df.columns)
print('constructors_raw_df', constructors_raw_df.columns)
print('constructor_standings_raw_df', constructor_standings_raw_df.columns)
print('driver_standings_raw_df', driver_standings_raw_df.columns)
print('drivers_raw_df', drivers_raw_df.columns)
print('laptimes_raw_df', laptimes_raw_df.columns)
print('pitstops_raw_df', pitstops_raw_df.columns)
print('qualifying_raw_df', qualifying_raw_df.columns)
print('races_raw_df', races_raw_df.columns)
print('results_raw_df', results_raw_df.columns)
print('seasons_raw_df', seasons_raw_df.columns)
print('sprint_results', sprint_results.columns)
print('status_raw_df', status_raw_df.columns)

circuits_raw_df Index(['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng',
       'alt', 'url'],
      dtype='object')
constructors_raw_df Index(['constructorId', 'constructorRef', 'name', 'nationality', 'url'], dtype='object')
constructor_standings_raw_df Index(['constructorStandingsId', 'raceId', 'constructorId', 'points',
       'position', 'positionText', 'wins'],
      dtype='object')
driver_standings_raw_df Index(['driverStandingsId', 'raceId', 'driverId', 'points', 'position',
       'positionText', 'wins'],
      dtype='object')
drivers_raw_df Index(['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob',
       'nationality', 'url'],
      dtype='object')
laptimes_raw_df Index(['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds'], dtype='object')
pitstops_raw_df Index(['raceId', 'driverId', 'stop', 'lap', 'time', 'duration',
       'milliseconds'],
      dtype='object')
qualifying_raw_df Index(['qualifyId', 'raceId', 'driverId',

## Merging data
Note: not filtering for drivers

In [11]:
# # Process drivers data set

# drivers = drivers_raw_df[['driverId','forename','surname','number']].reset_index(drop=True)
# # Make name column
# drivers['Name'] = drivers[['forename', 'surname']].agg(' '.join, axis=1)
# drivers.drop(['forename','surname'], axis=1, inplace=True)

# driver_name_id_map = dict(zip(drivers['Name'], drivers['driverId']))
# def get_driverID(name):
#     return drivers.loc[drivers['Name']==name, 'driverId'].squeeze()

# driver_name_number_map = dict(zip(drivers['Name'], drivers['number']))
# def get_driver_number(name):
#     return drivers.loc[drivers['Name']==name, 'number'].squeeze()

# driver_names_of_interest = ['Max Verstappen',
#                            'Sergio Pérez',
#                            'Lewis Hamilton',
#                            'Fernando Alonso',
#                            'Charles Leclerc',
#                            'Lando Norris',
#                            'George Russell',
#                            'Oscar Piastri',
#                            'Lance Stroll',
#                            'Pierre Gasly',
#                            'Esteban Ocon',
#                            'Alexander Albon',
#                            'Nico Hülkenberg',
#                            'Valtteri Bottas',
#                            'Guanyu Zhou',
#                            'Yuki Tsunoda',
#                            'Kevin Magnussen',
#                            #'Liam Lawson',
#                            'Logan Sargeant',
#                            'Nyck de Vries',
#                            'Daniel Ricciardo']
# driver_ids_of_interest = [driver_name_id_map[name] for name in driver_names_of_interest]

# driver_numbers_of_interest = [driver_name_number_map[name] for name in driver_names_of_interest]


In [94]:
# Driver name
drivers = drivers_raw_df[['driverId','forename','surname','number']].reset_index(drop=True)
drivers['Name'] = drivers[['forename', 'surname']].agg(' '.join, axis=1)
drivers.drop(['forename','surname'], axis=1, inplace=True)


# start with race results
# extend positionText labels (see EDA)
results_raw_df['positionText'] = results_raw_df['positionText'].replace({'R':'Retired','D':'Disqualified','N':'not classified', 'E':'Excluded','W':'Withdrawn','F':'failed to qualify'})
# drop time, milliseconds, fastestLap, rank, fastestLapTime, fastestLapSpeed, and number (http://ergast.com/mrd/methods/results/)
races = results_raw_df.drop(['time','milliseconds','fastestLap','rank','fastestLapTime','fastestLapSpeed','number'], axis=1)
races.drop(['positionOrder'], axis=1, inplace=True)
races['position'].fillna('NaN', inplace=True)
#display(races.isna().sum())
assert races.isna().sum().sum() == 0, 'there are null values'

# Get race data (ignoring qualifying data)
races_raw_df.rename(columns={'name':'Race'}, inplace=True)
prev_len = races.shape[0]
races = races.merge(races_raw_df[['raceId','Race','round','circuitId','date','url']], on='raceId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0

# Explode date
races['date'] = pd.to_datetime(races['date'])
races['Month'] = races['date'].dt.month
races['Year'] = races['date'].dt.year
#races.drop(['date'], axis=1, inplace=True)

# Filter year due to bad quality data in early years (multiple results reported for a driver in a race) -- see EDA for details
races = races[races['Year']>1964].reset_index(drop=True)
assert races.groupby(['raceId','driverId'])['position'].nunique().max()==1, 'there are multiple positions reported for a driver in a race'

# Merge with drivers
prev_len = races.shape[0]
races = races.merge(drivers[['driverId','Name']], on='driverId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races['Name'].isna().sum() == 0


# # Filter for drivers of interest
# races = races[races['Name'].isin(driver_names_of_interest)].reset_index(drop=True)
# assert races['driverId'].nunique() == len(driver_ids_of_interest), 'confirm we have lap data for all drivers of interest'
# #races.drop(['driverId'], axis=1, inplace=True)
# assert races.groupby(['raceId','Name']).size().max() == 1, 'driver occurs twice in a race'
# assert races.groupby(['raceId','driverId'])['position'].nunique().max() == 1


# Get status string
prev_len = races.shape[0]
races = races.merge(status_raw_df[['statusId','status']], on='statusId', how='left')
assert races['status'].isna().sum() == 0
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
races.drop(['statusId'], axis=1, inplace=True)


# Get car name
prev_len = races.shape[0]
constructors_raw_df.rename(columns={'name':'Car'}, inplace=True)
car_names = constructors_raw_df['Car'].unique()
races = races.merge(constructors_raw_df[['constructorId','Car']], on='constructorId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races['Car'].isna().sum() == 0
races.drop(['constructorId'], axis=1, inplace=True)


# Get circuit data
prev_len = races.shape[0]
circuits_raw_df.rename(columns={'name':'Circuit'}, inplace=True)
circuits_raw_df['alt'].fillna(circuits_raw_df['alt'].median(), inplace=True) # impute median altitude -- #todo may want to change
races = races.merge(circuits_raw_df[['circuitId','Circuit','location','country','alt','lat','lng']], on='circuitId', how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0
races.drop(['circuitId'], axis=1, inplace=True)

# Get number of laps in the race
number_of_laps = results_raw_df[results_raw_df['statusId']==1].groupby(['raceId'])['laps'].max().reset_index(name='nlaps')
assert races.isna().sum().sum() == 0, 'there is at least one race where none of the drivers finished -- cannot find number of laps in the race'
prev_len = races.shape[0]
races = races.merge(number_of_laps, on='raceId',how='left')
assert races.shape[0] == prev_len, 'confirm no duplicates formed during merge'
assert races.isna().sum().sum() == 0, 'there is at least one race where none of the drivers finished -- cannot find number of laps in the race'
races.drop(['laps'], axis=1, inplace=True)



# Summary
print('First year:',  races['Year'].min())
print('# Races:', races['raceId'].nunique())

First year: 1965
# Races: 960


In [5]:
# # Race Model Filters

# # filter for races with 32 or fewer drivers
# prev_n_races = races['raceId'].nunique()

# valid_races = races.groupby('raceId').size()
# valid_races = valid_races[valid_races<=32].index
# races = races[races['raceId'].isin(valid_races)].reset_index(drop=True)
# print('Races removed:', prev_n_races - races['raceId'].nunique(), '({} races now)'.format(races['raceId'].nunique()))



In [13]:
## Dynamic race model filters

# when number of race laps < k + threshold
k = 10
threshold = k*2
prev_n_races = races['raceId'].nunique()
race_counts = races['raceId'].value_counts()
valid_races = race_counts[race_counts > 10].index
races = races[races['raceId'].isin(valid_races)].reset_index(drop=True)
print('Races removed:', prev_n_races - races['raceId'].nunique())

Races removed: 0


In [14]:
print('unique races', races['raceId'].nunique())

unique races 960


## Construct Model Data With Lap Data

In [72]:
# adds padding rows to races with less than 24 drivers
def pad_race(group, target_rows=24, invalid_finishing_position_num=25):
    current_rows = group.shape[0]
    # If the current number of rows is less than the target, pad the group
    if current_rows < target_rows:
        # Calculate the number of rows to append
        rows_to_add = target_rows - current_rows
        # Create a DataFrame with the same columns, filled with NaNs
        padding_df = pd.DataFrame(np.nan, index=range(rows_to_add), columns=group.columns)
        # Assign the raceId to the padding DataFrame
        padding_df['raceId'] = group['raceId'].iloc[0]
        padding_df['driverId'] = -1
        padding_df['y'] = invalid_finishing_position_num
        if 'coef' in padding_df.columns:
            padding_df['coef'] = group['coef'].max() + group['coef'].std()
            #print('padding_df', padding_df['coef'].max())

        # Append the padding DataFrame to the group
        group = pd.concat([group, padding_df], ignore_index=True)
    return group

# Replace race driver order with random order with num_copies copies
def randomize_race(group, num_copies=4):
    randomized_races = []
    race_id = group['raceId'].iloc[0]

    # Create each random copy
    for i in range(num_copies):
        randomized_group = group.sample(frac=1, replace=False, random_state=i+1).reset_index(drop=True)
        randomized_group['copy_id'] = f"{race_id}_{i}"  # Create a unique ID for each copy
        randomized_races.append(randomized_group)
    
    # Concatenate
    return pd.concat(randomized_races, ignore_index=True)

def train_test_split_by_id(df, id_column, train_size=0.8):
    unique_ids = df[id_column].unique()
    
    train_ids, temp_ids = train_test_split(unique_ids, train_size=train_size, random_state=100)

    test_ids, valid_ids = train_test_split(temp_ids, train_size=0.5, random_state=100)

    train_df = df[df[id_column].isin(train_ids)]
    valid_df = df[df[id_column].isin(valid_ids)]
    test_df = df[df[id_column].isin(test_ids)]
    
    return train_df, valid_df, test_df

def pivot_model_data(df):

    # value columns
    laps_and_y_cols = [col for col in df.columns if 'lap_' in col or col == 'y' or col == 'coef']

    pivot = df.pivot_table(index='copy_id', 
                        columns=['model_driverId'], 
                        values=laps_and_y_cols, fill_value=np.nan)

    # Flatten the MultiIndex in columns
    #pivot.columns = [f'model_driver{model_driver_id}_{lap_id}pos' if lap_id else 'copy_id' for model_driver_id, lap_id in pivot.columns]
    new_columns = []
    for col in pivot.columns.tolist()[:]:
        # Concatenate the model_driverId with the lap information
        driver_number = 'driver' + str(col[1])  # Increment model_driverId by 1 to start numbering from 1
        lap_info = col[0]
        new_columns.append(f'{driver_number}_{lap_info}')
    pivot.columns = new_columns
    pivot.reset_index(inplace=True, drop=False)
    return pivot

In [88]:
def make_model_data(laptimes_raw_df, races, laps_to_consider, laps_text, path, coef=False, race_copies=3, metric='milliseconds'):
    # get laptimes (note: all races in laptimes_raw_df are in races dataframe)
    laptimes = laptimes_raw_df[laptimes_raw_df['raceId'].isin(races['raceId'].unique())].reset_index(drop=True)[['raceId','driverId','lap','position','milliseconds']]
    n_drivers = laptimes.groupby(['raceId'])['driverId'].nunique().max()
    print('max number drivers per race', n_drivers)

    # calculate driver finishing times
    driver_finishing_times = laptimes.groupby(['raceId','driverId'])['milliseconds'].sum().reset_index(name='finishing_time') # todo: fix where driver doesn't finish

    # filter data so that it corresponds to desired feature set
    laptimes = laptimes[laptimes['lap'].isin(laps_to_consider)].reset_index(drop=True)

    laptimes['lap'] = 'lap_' + laptimes['lap'].astype(str).str.zfill(2)
    #print(laptimes['lap'].unique())

    # pivot lap data
    if metric == 'milliseconds':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=0)  # fill missing values with 0
    elif metric == 'position':
        model_data = laptimes.pivot_table(index=['raceId','driverId'], columns=['lap'], values=metric, fill_value=25)  # fill missing values with 25
    else:
        raise ValueError('metric must be milliseconds or position')

    model_data.reset_index(inplace=True)

    driver_feature = pd.read_csv('data/driver_feature_model3_coef.csv')
    model_data = model_data.merge(driver_feature[['driverId','coef']], on=['driverId'], how='left')
    model_data['coef'] = model_data['coef'].fillna(model_data['coef'].quantile(0.95)) # these are drivers that don't have a coef due to only being in 1 race -- impute with 95th percentile value

    # Finishing positions (Y)
    positions = races[['raceId','driverId','positionText']].drop_duplicates().reset_index(drop=True)
    model_position_text_map = {'Retired':'25','Disqualified':'25', 'not classified':'25','Excluded':'25'}

    positions['y'] = positions['positionText'].replace(model_position_text_map)
    positions.drop(['positionText'], axis=1, inplace=True)
    prev_len = model_data.shape[0]
    model_data = model_data.merge(positions, on=['raceId','driverId'], how='left')
    #display(model_data['Y'].value_counts())

    assert model_data.shape[0] == prev_len, 'duplicates formed during merge'
    assert model_data.groupby(['raceId','driverId'])['y'].nunique().max() == 1, 'each driver should have one finishing position'
    assert model_data['y'].isin(['Withdrawn','failed to qualify']).sum() == 0, 'should not have any drivers who withdrew or failed to qualify'
    model_data['y'] = model_data['y'].astype(int)
    assert model_data.groupby(['raceId'])['y'].min().max() == 1, 'races missing a winner'
    assert model_data.groupby(['raceId'])['y'].min().min() == 1, 'positions < 1'
    assert model_data.groupby(['raceId'])['y'].max().max() <= 25, 'race has position > 24'
    assert model_data.isna().sum().sum() ==0

    assert model_data.isna().sum().sum() == 0, 'there are null values'
    # add rows if number of drivers in a race is less than 24
    model_data = model_data.groupby('raceId').apply(pad_race).reset_index(drop=True)
    if metric == 'milliseconds':
        model_data.fillna(0, inplace=True)
    else:
        model_data.fillna(25, inplace=True)
        assert model_data['y'].min() == 1, 'positions < 1'

    

     # replace data with randomly ordered copies
    model_data = model_data.groupby('raceId', group_keys=False).apply(randomize_race, (race_copies)).reset_index(drop=True)
    # Create 'model_driverId' within each 'copy_id' group
    model_data['model_driverId'] = model_data.groupby('copy_id').cumcount() + 1

    train_df, valid_df, test_df = train_test_split_by_id(model_data, 'copy_id')

    train_df_mf = pivot_model_data(train_df).set_index('copy_id')
    valid_df_mf = pivot_model_data(valid_df).set_index('copy_id')
    test_df_mf = pivot_model_data(test_df).set_index('copy_id')

    n_features = n_drivers * len(laps_to_consider)
    if coef:
        n_features += n_drivers

    train_X, train_y = train_df_mf.iloc[:,:n_features], train_df_mf.iloc[:,n_features:]
    valid_X, valid_y = valid_df_mf.iloc[:,:n_features], valid_df_mf.iloc[:,n_features:]
    test_X, test_y = test_df_mf.iloc[:,:n_features], test_df_mf.iloc[:,n_features:]
    
    coef_text = 'coef' if coef else 'no_coef'

    #binary_str = 'binary' if binary_y else 'categorical'
    path = '{}/{}/{}/{}/'.format(path, metric, coef_text, laps_text)

    # make folder if doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    # print(train_X.shape, train_y.shape)
    # print(valid_X.shape, valid_y.shape)
    # print(test_X.shape, test_y.shape)
    
    train_X.to_csv('{}{}copies_X_train.csv'.format(path, race_copies), index=True)
    train_y.to_csv('{}{}copies_y_train.csv'.format(path, race_copies), index=True)
    valid_X.to_csv('{}{}copies_X_valid.csv'.format(path, race_copies), index=True)
    valid_y.to_csv('{}{}copies_y_valid.csv'.format(path, race_copies), index=True)
    test_X.to_csv('{}{}copies_X_test.csv'.format(path, race_copies), index=True)
    test_y.to_csv('{}{}copies_y_test.csv'.format(path, race_copies), index=True)

In [92]:
path = 'Data/RaceMultiOutputModelRandomized2/'

# delete everything in path
for root, dirs, files in os.walk(path):
    for file in files:
        os.remove(os.path.join(root, file))
    for dir in dirs:
        shutil.rmtree(os.path.join(root, dir))

# Baseline datasets
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1], laps_text='Lap1',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[2], laps_text='Lap2',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[3], laps_text='Lap3',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[4], laps_text='Lap4',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[5], laps_text='Lap5',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[6], laps_text='Lap6',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[7], laps_text='Lap7',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[8], laps_text='Lap8',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[9], laps_text='Lap9',
                race_copies=1, metric='position', path=path)
make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[10], laps_text='Lap10',
                race_copies=1, metric='position', path=path)



# Laps1to10. Laps1to5 datasets
for metric in ['position']:
    for coef in [True, False]:
    

        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1], laps_text='Lap1',
                coef=coef, race_copies=3, metric=metric, path=path)
        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[10], laps_text='Lap10',
                coef=coef, race_copies=3, metric=metric, path=path)
        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1,2,3,4,5], laps_text='Laps1to5',
                        coef=coef, race_copies=3, metric=metric, path=path)
        make_model_data(laptimes_raw_df=laptimes_raw_df, races=races, laps_to_consider=[1,2,3,4,5,6,7,8,9,10], laps_text='Laps1to10',
                        coef=coef, race_copies=3, metric=metric, path=path)
    
notify('done building datasets','')

max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
max number drivers per race 24
